In [ ]:
from To_nparray import generate_data, traintest_split
from keras.models import load_model
from model_evalution import evalution_metrics

In [ ]:
# path
data_output = 'Output/PC_6_712_354_712/'
model_output = 'Output/PC_6_712_354_712/Model'

In [ ]:
train_data,train_label = generate_data(pos_fasta='AMP.tr.fa', neg_fasta='DECOY.tr.fa')
test_data, test_label = generate_data(pos_fasta='AMP.te.fa', neg_fasta='DECOY.te.fa')
val_data, val_label = generate_data(pos_fasta='AMP.eval.fa', neg_fasta='DECOY.eval.fa')

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D
from keras.layers import LSTM,Add,Input
from keras.layers import LeakyReLU
from keras import models
from keras import optimizers
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
model_name = 'PC_6'
input_ = Input(shape=(200,6))
cnn = Conv1D(64 ,16,activation = 'relu', padding="same")(input_)
#post_cnn = LeakyReLU(alpha=0.1)(cnn)
lstm = LSTM(units=100,return_sequences=False)(cnn)
result = Dense(1, activation = "sigmoid")(lstm)
model = Model(inputs=input_,outputs=result)

model.compile(optimizer=optimizers.Adam(lr=3*1e-4),
                loss='binary_crossentropy',
                metrics=['accuracy'])

e_s = keras.callbacks.EarlyStopping(monitor='val_loss',
                                        min_delta=0,
                                        patience=80,
                                        verbose=0, mode='min')
best_weights_filepath = model_output+'/%s_best_weights.h5'%model_name
saveBestModel = keras.callbacks.ModelCheckpoint(best_weights_filepath, 
                                                    monitor='val_loss', 
                                                    verbose=1, 
                                                    save_best_only=True, 
                                                    mode='auto')
CSVLogger = keras.callbacks.CSVLogger(model_output+"/%s_csvLogger.csv"%model_name,separator=',', append=False)

t_m=model.fit(train_data,train_label,shuffle=True,validation_data=(val_data, val_label), 
                epochs=500, batch_size=int(1*len(train_data)),callbacks=[saveBestModel,CSVLogger])


In [ ]:
model = load_model('Output/PC_6_712_354_712/Model/PC_6_best_weights.h5')
labels_score = model.predict(test_data)
evalution_metrics(test_label, labels_score,save=True, txt_name='test', path = model_output)

In [ ]:
from keras.models import load_model 
ex_data, ex_labels =generate_data(pos_fasta='External_Test_Pos_565.fasta',neg_fasta='External_Test_Neg_565.fasta') 
ex_labels_score = model.predict(ex_data) 
evalution_metrics(ex_labels, ex_labels_score, save=True, txt_name='ex_test', path = model_output)